In [15]:
import sqlite3 as sq3
import pandas as pd
import requests
import json 
from pandas.io.json import json_normalize

In [17]:
#creat DB + load data
data = pd.read_csv("udemy_courses.csv")
db = sq3.connect("udemy_courses_db.db")
data.to_sql("udemy_courses", db, index = False)
cur = db.cursor()

In [42]:
#Q1 - TOP 5 free courses by subject
#is_paid column => True = 1, False = 0

cur.execute('SELECT * FROM\
            (SELECT "course_title", "course_id", "is_paid", "subject", "num_subscribers", ROW_NUMBER()\
            OVER (PARTITION BY "subject" ORDER BY "num_subscribers" DESC) AS rank\
            FROM "udemy_courses"\
            WHERE "is_paid" = 0)\
            WHERE rank < 6')
    
TOP5 = pd.DataFrame(cur.fetchall())
TOP5.columns = ["course_title", "course_id", "is_paid", "subject", "num_subscribers", "rank"]
TOP5 = TOP5.set_index("rank")
display(TOP5.head())

,course_title,course_id,is_paid,subject,num_subscribers
rank,,,,,
1,Bitcoin or How I Learned to Stop Worrying and ...,49798,0,Business Finance,65576
2,Accounting in 60 Minutes - A Brief Introduction,48841,0,Business Finance,56659
3,Stock Market Investing for Beginners,133536,0,Business Finance,50855
4,Introduction to Financial Modeling,151668,0,Business Finance,29167
5,Forex Basics,97338,0,Business Finance,22344


In [32]:
# Q2 - TOP10 most popular courses
cur.execute('SELECT "course_id", "course_title"\
            FROM "udemy_courses"\
            ORDER BY "num_subscribers" DESC\
            LIMIT 10')
    
TOP10 = pd.DataFrame(cur.fetchall())
TOP10.columns = ["course_id", "course_title"]
TOP10["rank"] = range(1, 11)
TOP10 = TOP10.set_index("rank")
display(TOP10)

,course_id,course_title
rank,,
1,41295,Learn HTML5 Programming From Scratch
2,59014,Coding for Entrepreneurs Basic
3,625204,The Web Developer Bootcamp
4,173548,Build Your First Website in 1 Week with HTML5 ...
5,764164,The Complete Web Developer Course 2.0
6,19421,Free Beginner Electric Guitar Lessons
7,473160,Web Design for Web Developers: Build Beautiful...
8,94430,Learn Javascript & JQuery From Scratch
9,130064,Practical PHP: Master the Basics and Code Dyna...


In [43]:
# Q3 - best year by number of courses

cur.execute('SELECT strftime("%Y","published_timestamp") AS "year", COUNT(*) AS "count"\
            FROM "udemy_courses"\
            GROUP BY "year"\
            ORDER BY "count" DESC')
    
best_year = pd.DataFrame(cur.fetchall())
best_year.columns = ["year", "count"]
display(best_year.head(1))

,year,count
0,2016,1206


In [44]:
# Q4 - highest price per lecture

cur.execute('SELECT "course_title", CAST("price" AS FLOAT) / "num_lectures" AS "cost per lecture USD"\
            FROM "udemy_courses"\
            WHERE "is_paid" = 1\
            ORDER BY "cost per lecture USD" DESC')
    
highest_price = pd.DataFrame(cur.fetchall())
highest_price.columns = ["course_title", "cost_per_lecture_USD"]
display(highest_price.head(1))

,course_title,cost_per_lecture_USD
0,Make tech circles in Adobe Illustrator in 30 m...,40.0


In [57]:
# Q5 - average price per lecture

cur.execute('SELECT AVG(CAST("price" AS FLOAT) / "num_lectures") AS "average price"\
            FROM "udemy_courses"\
            WHERE "is_paid" = 1')
    
avg_price = cur.fetchall()[0][0]
print(round(avg_price, 2))

2.86


In [59]:
# Q6 - max_price per monte + year

cur.execute('SELECT "course_title", MAX("price") AS "max price", strftime("%Y-%m","published_timestamp") AS "date"\
            FROM "udemy_courses"\
            GROUP BY strftime("%Y %m","published_timestamp")')
    
max_price = pd.DataFrame(cur.fetchall())
max_price.columns = ["course_title", "max_price", "date"]
display(max_price)

,course_title,max_price,date
0,Javascript for Beginners,20,2011-07
1,HTML Tutorial: HTML & CSS for Beginners,20,2011-09
2,Become a Web Developer from Scratch,120,2011-11
3,Become a Certified Web Developer,100,2011-12
4,Aprende a Tocar Teclado - Piano,35,2012-01
...,...,...,...
65,The Almost Perfect Options Trading Strategy Sy...,200,2017-03
66,The Only Investment Strategy You Need For Your...,200,2017-04
67,How I Learned Stock Trading (and How You Can Too),200,2017-05
68,Winning Forex Trading with Live Forex Trading ...,200,2017-06


In [18]:
# Getting rating info for first 500 courses from Udemy using REST API

course_id = data["course_id"].to_list()

with open("Authorization") as file:
    Authorization = file.read()
    
dfs = []
first_500 = []

for i in course_id[:501]:
    URL = "https://www.udemy.com/api-2.0/courses/{}/reviews/".format(i)
    response = requests.get(URL, headers={'Authorization': Authorization})
    if (("results" in response.json()) and ("rating" in json_normalize(data = response.json(), record_path = "results").columns)):
        dfs.append(json_normalize(data = response.json(), record_path = "results")["rating"])
        first_500.append(i)

rating = pd.concat(dfs, keys = first_500).droplevel(-1).reset_index()
rating.columns = ["course_id", "rating_score"]
rating.to_sql("rating", db, index = False)

In [41]:
# Q7 - TOP 10 courses with the highest avarage rating score

cur.execute('SELECT course_id, AVG(rating_score) AS Avarage\
            FROM rating\
            GROUP BY course_id\
            ORDER BY Avarage DESC\
            LIMIT 10')
    
TOP10_avg = pd.DataFrame(cur.fetchall())
TOP10_avg.columns = ["course_id", "avg_score"]
TOP10_avg

,course_id,avg_score
0,1208148,5.0
1,1134864,5.0
2,848664,5.0
3,811534,5.0
4,521234,5.0
5,429482,5.0
6,429308,5.0
7,73938,5.0
8,920424,4.9
9,722174,4.9
